# Fit Exposure MF with exposure covariantes (Location ExpoMF) to the Gowalla dataset

In [10]:
%load_ext autoreload
%autoreload 2
import glob
import os
# if you are using OPENBLAS, you might want to turn this option on. Otherwise, joblib might get stuck
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse
import pandas as pd
from numpy import zeros

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Anaconda3\envs\py2\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [11]:
#Change this to suit your need before running
FOLDER_NAME = 'gowalla_pro'
COUNT_NAME = '/USERCOUNT_6000_MINSC_30'
DATA_ROOT = '../' + FOLDER_NAME
ID_DATA_ROOT = DATA_ROOT + COUNT_NAME
num_features = 100

In [12]:
unique_uid = list()
with open(os.path.join(ID_DATA_ROOT, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
    
unique_sid = list()
with open(os.path.join(ID_DATA_ROOT, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

In [13]:
n_songs = len(unique_sid)
n_users = len(unique_uid)
print n_songs, n_users

1298 3666


### Load the data and train the model

In [14]:
def load_data(csv_file, shape=(n_users, n_songs)):
    tp = pd.read_csv(csv_file)    
    rows, cols = np.array(tp['uid'], dtype=np.int32), np.array(tp['sid'], dtype=np.int32)
    count = tp['rating']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

In [15]:
train_data, rows, cols = load_data(os.path.join(ID_DATA_ROOT, 'train.num.csv'))
# binarize the data, setting all data equal 1
train_data.data = np.ones_like(train_data.data)
vad_data, rows_vad, cols_vad = load_data(os.path.join(ID_DATA_ROOT, 'vad.num.csv'))
# binarize the data
vad_data.data = np.ones_like(vad_data.data)
test_data, rows_test, cols_test = load_data(os.path.join(ID_DATA_ROOT, 'test.num.csv'))
# binarize the data
test_data.data = np.ones_like(test_data.data)

`feat_venue_locs.tsv` contains the location features (part of the [pre-processed data](http://dawenl.github.io/data/gowalla_pro.zip)), which are generated in the following way: 
- Run GMM (from [scikit.learn](http://scikit-learn.org/)) on all the venue locations.
- For each venue, take the expected cluster assignment as location features `pi`.

In [16]:
# filtering out locations that should have been filtered out. see filter_triplets
pi = np.loadtxt(os.path.join(DATA_ROOT, 'feat_venue_locs.tsv'), dtype='float32')
mask = np.ones(pi.shape, dtype=bool)
for i in range(0, pi.shape[0]):
    if "%d" % pi[i, 0] not in unique_sid:
        mask[i,:] = False
pi = pi[mask,...]
pi = pi.reshape(-1, num_features+1)


# sanity check to make sure all the venues has its corresponding feature    
for i, s in enumerate(unique_sid):
    if s != "%d" % pi[i, 0]:
        print i, s, pi[i, 0]
        break
# the first column of pi is sid
# the first column is ID, don't need them
pi = pi[:, 1:]

In [17]:
import paral_gibbs3_expomf_cov
import gibbs3_expomf_cov
n_components = 100
max_iter = 50
n_jobs = 1
lam = 1e-5
# here we use the best performing init_mu from per-item \mu_i experiment
init_mu = 0.01
max_epoch = 10

save_dir=FOLDER_NAME + "Paral_gibb3_ExpoMF_params_K%d_lam%1.0E_initmu%1.0E_maxepoch%d" % (n_components, lam, init_mu, max_epoch)

#coder = expomf_cov.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=1000, 
#                          batch_sgd=10, max_epoch=max_epoch, init_std=0.01,
#                          n_jobs=n_jobs, random_state=98765, save_params=True, save_dir=save_dir, 
#                          early_stopping=True, verbose=True, 
#                          lam_y=1., lam_theta=lam, lam_beta=lam, lam_nu=lam, init_mu=init_mu, learning_rate=.5)
import paral_gibbs3_expomf_cov
coder = paral_gibbs3_expomf_cov.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=10000, 
                          batch_sgd=100, max_epoch=max_epoch, init_std=0.01,
                          n_jobs=n_jobs, random_state=98765, save_params=True, save_dir=save_dir, 
                          early_stopping=True, verbose=True, 
                          lam_y=1., lam_theta=lam, lam_beta=lam, lam_nu=lam, init_mu=init_mu, learning_rate=.5, debugCov = True)

In [ ]:
#parallel
import paral_gibbs3_expomf_cov
n_jobs = -1
coder = paral_gibbs3_expomf_cov.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=10000, 
                          batch_sgd=100, max_epoch=max_epoch, init_std=0.01,
                          n_jobs=n_jobs, random_state=98765, save_params=True, save_dir=save_dir, 
                          early_stopping=True, verbose=True, 
                          lam_y=1., lam_theta=lam, lam_beta=lam, lam_nu=lam, init_mu=init_mu, learning_rate=.5, debugCov=True)
para_dir = 'gowalla_proParal_gibb3_ExpoMF_params_K100_lam1E-05_initmu1E-02_maxepoch10/_iter9.npz'
coder.fit(train_data, pi, para_dir, init_only_mu=False, random_mu = True)

Start to sample...
Iteration: #0
	Sampling exposure covariate: time=0.89
	Sampling user factors...[  5.97761202e+04  -7.45393477e+02  -4.47580123e+02   3.40737546e+02
  -1.96931337e+03  -2.47718843e+02   1.75716916e+03   8.64062965e+01
   1.27354460e+03  -1.32524780e+03   2.37139746e+01   2.99693230e+02
  -2.85824803e+02  -1.76770031e+03  -1.09582304e+03   6.38783022e+02
  -1.75793419e+03   8.16401683e+02  -1.11564534e+03  -8.03272321e+02
   6.60182578e+02  -3.47220740e+02  -4.93362470e+02   8.29269283e+02
  -3.75647421e+02   6.67408769e+02  -2.16676307e+02  -6.63342916e+02
  -2.99284454e+03  -1.37145284e+03  -4.66139258e+02   6.09092324e+02
  -1.25195827e+03   6.75182537e+02  -5.89762551e+02  -1.68991979e+03
  -1.86679869e+03  -5.84280741e+02  -1.67496018e+03   1.27593983e+03
   4.49540070e+02  -5.90544561e+02   3.44470366e+02   1.88475552e+03
  -4.12567105e+02   5.96403079e+02  -6.27619523e+02  -8.44495564e+02
  -7.57943337e+02   1.28461216e+03  -4.80847726e+02  -1.57082575e+03
  -8.

In [ ]:
#non parallel
para_dir = '/home/ec2-user/notebook/expo-mf/src/YELP_20000USERS_100_feature_restaurant_only_location_only_Numeric_Id100_lxam1E-05_initmu1E-02_maxepoch10/ExpoMF_cov_K100_mu1.0e-02_iter12.npz'
coder.fit(train_data, pi, para_dir, init_only_mu=False, random_mu = False)

In [ ]:
#non parallel
para_dir = '/home/ec2-user/notebook/expo-mf/src/YELP_20000USERS_100_feature_restaurant_only_location_only_Numeric_Id100_lxam1E-05_initmu1E-02_maxepoch10/ExpoMF_cov_K100_mu1.0e-02_iter12.npz'
coder.fit(train_data, pi, para_dir, init_only_mu=False, random_mu = True)

It seems that after a few epochs the validation loss will not decrease. However, we empirically found that it is still better to train for more epochs, instead of stop the SGD

## Evaluate the performance on heldout testset

In [19]:
n_params = len(glob.glob(os.path.join(save_dir, '*.npz')))

# params = np.load(os.path.join(save_dir, 'ExpoMF_cov_K%d_mu%.1e_iter%d.npz' % (n_components, init_mu, 0)))
params = np.load(os.path.join(save_dir, '_iter9.npz'))

U, V, nu, alpha = params['U'], params['V'], params['nu'], params['alpha']

### Rank by $\mathbb{E}[y_{ui}] = \mu_{ui}\theta_u^\top\beta_i$

In [26]:
for i in range(11):
    n_params = len(glob.glob(os.path.join(save_dir, '*.npz')))

    # params = np.load(os.path.join(save_dir, 'ExpoMF_cov_K%d_mu%.1e_iter%d.npz' % (n_components, init_mu, 0)))
    params = np.load(os.path.join(save_dir, '_iter' + str(i)+ '.npz'))

    U, V, nu, alpha = params['U'], params['V'], params['nu'], params['alpha']
    import rec_eval
    mu = {'params': [nu, pi, alpha], 'func': gibbs3_expomf_cov.get_mu}

#     print 'Test Recall@20: %.4f' % rec_eval.recall_at_k(train_data, test_data, U, V, k=20, mu=mu, vad_data=vad_data)
#     print 'Test Recall@50: %.4f' % rec_eval.recall_at_k(train_data, test_data, U, V, k=50, mu=mu, vad_data=vad_data)
    print 'Test NDCG@100: %.4f' % rec_eval.normalized_dcg_at_k(train_data, test_data, U, V, k=100, mu=mu, vad_data=vad_data)
#     print 'Test MAP@100: %.4f' % rec_eval.map_at_k(train_data, test_data, U, V, k=100, mu=mu, vad_data=vad_data)

Test NDCG@100: 0.0193
Test NDCG@100: 0.0177
Test NDCG@100: 0.0218
Test NDCG@100: 0.0198
Test NDCG@100: 0.0232
Test NDCG@100: 0.0198
Test NDCG@100: 0.0190
Test NDCG@100: 0.0196
Test NDCG@100: 0.0229
Test NDCG@100: 0.0192
Test NDCG@100: 0.0215
